In [5]:
#transforming a pandas series into a dataframe
df_courses = pd.DataFrame(df_courses)

In [6]:
# changing the format here in order to be able to compare the input with the dataframe
df_courses['Course Name'] = df_courses['Course Name'].str.replace('-', '')
#removing the msc from the course names 
df_courses['Course Name'] = df_courses['Course Name'].str.replace('msc', '')
df_courses['Course Name'] = df_courses['Course Name'].str.replace('Msc', '')
df_courses['Course Name'] = df_courses['Course Name'].str.replace('MSc','')
df_courses['coursename_clean'] = df_courses['Course Name'].str.lower().str.split()
df_courses['uni_name_clean'] = df_courses['University Name'].str.lower()
df_courses['city_lowered'] = df_courses['City'].str.lower()


df_courses

NameError: name 'df_courses' is not defined

In [51]:
all_words_set = set()

# Function to extract words from a course name and add them to the set
def extract_words(course_name):
    words = course_name.split()
    all_words_set.update(words)

# Apply the function to each row in the 'course name' column
df_courses['Course Name'].apply(extract_words)


print(all_words_set)

{'Design', 'Precision', 'FIFA', '(Hydrographic', 'Animate', '(Palaeoanthropology)', '(Waste,', 'Management', 'South', 'Hydrology', 'Nurse', 'Practice)', 'Planning)', 'Extremist', 'Implant', 'English', 'year)', 'programme)', 'Campus,', 'Accreditation)', '(Parttime', 'PGCert/PGDip/MSc', 'Crime,', '(Grenoble)', 'Inflammation', '(Consumer', 'HEMS', 'Yacht', 'Manufacture', 'validation)', 'Biochemistry)', '(Cellular', 'Insurance', 'Migrations', 'Chemistry', 'Collaboration', 'Trading', 'Needs', 'Earthquake', 'Programmable', '(Health', 'Disorders', 'Adaptation', 'Haematology', '(Online)', 'Territories', 'Games', 'Chronic', 'Failure', '(LPC)', 'Clinical', 'Infection,', 'East', 'Engineers', 'Urbanism', 'MHS/MSc', 'Regulatory', 'Nutrition,', '(Policy', 'General', 'Renewable', 'Elite', '(Distance', 'Wilderness', 'online', '(Minimallyinvasive', 'Optical', 'Wound', 'Aeronautics,', 'Policy,', '(Infection', 'Additive', 'ENGINEERING', 'Microsystems', 'Administration', 'Apprenticeship', '(M.A.,', 'place

In [52]:
vocabulary = {}
unique_id = 1
for word in list(all_words_set):
  vocabulary[unique_id] = word.lower()
  unique_id += 1

In [33]:
# Loop through the keys of the vocabulary
inverted_index = dict()
for i in list(vocabulary.keys()):
    inverted_index[i] = []
    for index, row in df_courses.iterrows():
        if vocabulary[i] in row['coursename_clean']:
            inverted_index[i].append(index)

In [53]:

def search_documents(split_cn):
    result_indices = set()
    for term in split_cn:
        if term in vocabulary.values():
            term_id = [key for key, value in vocabulary.items() if value == term][0]
            result_indices.update(inverted_index[term_id])
    return result_indices


In [ ]:
month_to_number = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Function to convert month names to numerical values
def month_to_number_converter(month_name):
    return month_to_number.get(month_name)

# Convert the "Start Date" column to numerical values in order to make a comparison
df_courses['Start Month'] = df_courses['Start Date'].apply(month_to_number_converter)


In [4]:
import pandas as pd

def search_eng():
    # Collect user inputs
    course_name = input('Enter Course Name (press enter to skip): ').lower()
    uni_name = input('Enter University Name (press enter to skip): ').lower()
    city = input('Enter City Name (press enter to skip): ').lower()
    online = input('Do you need an online modality? Reply with Y or N (press enter to skip): ').lower()

    # Default values for price range
    min_price = input('Enter Minimum Price (press enter to skip): ')
    max_price = input('Enter Maximum Price (press enter to skip): ')
    
    # Apply filters based on user inputs
    filtered_courses = df_courses.copy()
    
    today = datetime.now()

    filtered_courses = filtered_courses[filtered_courses['Start Month'] <= today.month]

# Sort the DataFrame by the courses that have already started 
    filtered_courses= filtered_courses.sort_values(by='Start Month')


    filtered_courses = filtered_courses.drop(columns=['Start Month'])

    if course_name:
        course_name_split = set(course_name.split())
        course_indices = search_documents(course_name_split)
        filtered_courses = filtered_courses.loc[list(course_indices)]

    if uni_name:
        uni_name_lower = uni_name.lower()
        filtered_courses = filtered_courses[filtered_courses['uni_name_clean'].str.contains(uni_name_lower)]

    if city:
        city_lower = city.lower()
        filtered_courses = filtered_courses[filtered_courses['city_lowered'].str.contains(city_lower)]

  
    filtered_courses['fees (£)'] = filtered_courses['fees (£)'].apply(lambda x: x if pd.notna(x) and str(x).replace('.', '', 1).isdigit() else 'Refer to university website for fees')

    # Filter based on price range
    if min_price and max_price:
        # Exclude courses with 'Refer to university website for fees' if user provides a fee range
        filtered_courses = filtered_courses[(filtered_courses['fees (£)'] != 'Refer to university website for fees') & (filtered_courses['fees (£)'] >= int(min_price)) & (filtered_courses['fees (£)'] <= int(max_price))]
    elif min_price:
        filtered_courses = filtered_courses[(filtered_courses['fees (£)'] != 'Refer to university website for fees') & (filtered_courses['fees (£)'] >= int(min_price))]
    elif max_price:
        filtered_courses = filtered_courses[(filtered_courses['fees (£)'] != 'Refer to university website for fees') & (filtered_courses['fees (£)'] <= int(max_price))]

    if online == 'y':
        filtered_courses = filtered_courses[filtered_courses['administration'].str.contains('online', case=False)]
    elif online == 'n':
        filtered_courses = filtered_courses[~filtered_courses['administration'].str.contains('online', case=False)]

  
    if not filtered_courses.empty:
        result_columns = ['Course Name', 'University Name', 'Link']
        print("Filtered courses:")
        print(filtered_courses[result_columns])
    else:
        print("No courses match the specified criteria.")


search_eng()


NameError: name 'df_courses' is not defined